In [ ]:
import os
import xarray as xr
import numpy as np
from datetime import datetime, timedelta

In [ ]:
def cut_netcdf_to_bounding_box(input_file, output_file, lonlatbox):
    """
    Cut a NetCDF file to a specified lon-lat bounding box and save it as a new file.

    Parameters:
        input_file (str): Path to the input NetCDF file.
        output_file (str): Path to save the output NetCDF file.
        lonlatbox (list): Bounding box in the format [lon_min, lon_max, lat_min, lat_max].

    Returns:
        None
    """
    # Unpack the lon-lat bounding box
    lon_min, lon_max, lat_min, lat_max = lonlatbox

    # Open the input NetCDF file
    ds = xr.open_dataset(input_file)

    # Select the region within the bounding box
    ds_cut = ds.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))

    # Upscale dimensions
    new_dims = {dim: ds_cut.sizes[dim] * 4 for dim in ds_cut.dims}

    # Create new coordinates
    new_coords = {coord: np.linspace(ds_cut[coord][0], ds_cut[coord][-1], size)
                  for coord, size in new_dims.items()}

    # Perform interpolation
    ds_upscaled = ds_cut.interp(coords=new_coords)
    # Save the new NetCDF file
    ds_upscaled.to_netcdf(output_file)

    print(f"New NetCDF file saved as: {output_file}")

    return ds_upscaled


In [ ]:
def get_files(directory, format):
    """
    Get a list of all GRIB2 files in the specified directory.

    Parameters:
    - directory: Path to the directory containing the GRIB2 files.
    - format: either '.grb2' or '.nc'
    Returns:
    - List of file paths to the GRIB2 files.
    """
    files = []
    for file_name in os.listdir(directory):
        if file_name.endswith(format):
            file_path = os.path.join(directory, file_name)
            files.append(file_path)
    return files

In [ ]:
def upscale_grid(nc_file):
    # Open the NetCDF file
    ds = xr.open_dataset(nc_file)

    # Upscale dimensions
    new_dims = {dim: ds.sizes[dim] * 4 for dim in ds.dims}

    # Create new coordinates
    new_coords = {coord: np.linspace(ds[coord][0], ds[coord][-1], size)
                  for coord, size in new_dims.items()}

    # Perform interpolation
    upscaled_data = ds.interp(coords=new_coords)

    return upscaled_data

In [ ]:
# Defined constants based on today's date and not required as input from the user
today = datetime.today()
year1 = today.year
month1 = today.month
dayofmonth = today.day
day1 = today.replace(day=1).strftime('%Y%m%d') # is this always just 01 or a date??

eventtime = today.replace(day=1).strftime('%Y-%m-%d')
date = today.strftime('%Y%m%d')

In [ ]:
# Constants
# Defines box encompassing the entire Great Lakes Basin
lonlatbox = [-110,-65,30,70]

# Which daily CFS run cycles would we like to download, process, and add to our csv basin files
utc = ['00','06','12','18']

In [ ]:
# Inputs from the user
download_dir = 'C:/Users/fitzpatrick/Desktop/Data/20240528/'

In [ ]:
file_list = get_files(download_dir, '.nc')

for file in file_list:
    #split the filename 
    name, ext = os.path.splitext(file)
    # New filename
    new_filename = name + '_upscaled' + ext
    
    new_netcdf = cut_netcdf_to_bounding_box(file, new_filename, lonlatbox)